In [2]:
from functools import reduce
from typing import List

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model

from interact.fields import FieldsManager, SparseField
from interact.layers import SparseLinear, AddBias
from interact.models import AFM
from interact.utils import to_sequences

from utils import get_dataset, DataSet, cost, MercariTranformer

In [3]:
train = get_dataset(DataSet.Train)
valid = get_dataset(DataSet.Valid)

In [4]:
mercari_transformer = MercariTranformer(
    CountVectorizer(min_df=10),
    CountVectorizer(min_df=10),
)

In [5]:
%%time
mercari_transformer.fit(pd.concat([train, valid], axis=0, ignore_index=True))

CPU times: user 59.1 s, sys: 1.21 s, total: 1min
Wall time: 1min


In [6]:
%%time
d_train = mercari_transformer.transform(train)

CPU times: user 1min 58s, sys: 662 ms, total: 1min 59s
Wall time: 1min 59s


In [7]:
%%time
d_valid = mercari_transformer.transform(valid)

CPU times: user 21.7 s, sys: 4.41 ms, total: 21.7 s
Wall time: 21.7 s


In [8]:
f_name = SparseField(
    name='name', 
    vocabulary_size=16543,
    m=10, 
    d=5,
)

f_description = SparseField(
    name='description', 
    vocabulary_size=30164,
    m=169, 
    d=5,
)

f_brand = SparseField(
    name='brand', 
    vocabulary_size=4658,
    m=1, 
    d=5,
)

f_condition = SparseField(
    name='item_condition_id', 
    vocabulary_size=5,
    m=1, 
    d=5,
)

f_shipping = SparseField(
    name='shipping', 
    vocabulary_size=2,
    m=1, 
    d=5,
)

f_category = SparseField(
    name='category', 
    vocabulary_size=946,
    m=5, 
    d=5,
)

In [9]:
fields = [
    f_name,
    f_description,
    f_brand,
    f_condition,
    f_shipping,
    f_category
]

In [10]:
afm = AFM(fields, l2_penalty=0.001)

In [11]:
afm.compile(optimizer='sgd', loss='mse')

In [12]:
train_inputs = []
train_inputs.append(to_sequences(d_train["name"], 10))
train_inputs.append(to_sequences(d_train["item_description"], 169))
train_inputs.append(to_sequences(d_train["brand_name"], 1))
train_inputs.append((d_train["item_condition_id"].argmax(axis=1) + 1).reshape((-1, 1)))
train_inputs.append((d_train["shipping"] + 1).values.reshape((-1, 1)))
train_inputs.append(d_train["category_name"])

In [13]:
valid_inputs = []
valid_inputs.append(to_sequences(d_valid["name"], 10))
valid_inputs.append(to_sequences(d_valid["item_description"], 169))
valid_inputs.append(to_sequences(d_valid["brand_name"], 1))
valid_inputs.append((d_valid["item_condition_id"].argmax(axis=1) + 1).reshape((-1, 1)))
valid_inputs.append((d_valid["shipping"] + 1).values.reshape((-1, 1)))
valid_inputs.append(d_valid["category_name"])

In [14]:
afm.fit(
    train_inputs, 
    np.log1p(train['price']),
    epochs=10,
    batch_size=32,
    shuffle=True,
    validation_data=(
        valid_inputs,
        np.log1p(valid['price'])
    ), 
    callbacks=[EarlyStopping()]
)

Train on 1082535 samples, validate on 200000 samples
Epoch 1/10


/home/sigmo/deep/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/sigmo/deep/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1082535/1082535 [==============================] - 174s 161us/sample - loss: 0.4317 - val_loss: 0.3501
Epoch 2/10
1082535/1082535 [==============================] - 165s 153us/sample - loss: 0.3217 - val_loss: 0.3053
Epoch 3/10
1082535/1082535 [==============================] - 152s 141us/sample - loss: 0.2975 - val_loss: 0.2941
Epoch 4/10
1082535/1082535 [==============================] - 142s 132us/sample - loss: 0.2913 - val_loss: 0.2911
Epoch 5/10
1082535/1082535 [==============================] - 142s 131us/sample - loss: 0.2897 - val_loss: 0.2905
Epoch 6/10
1082535/1082535 [==============================] - 150s 139us/sample - loss: 0.2893 - val_loss: 0.2903
Epoch 7/10
1082535/1082535 [==============================] - 166s 153us/sample - loss: 0.2892 - val_loss: 0.2898
Epoch 8/10
1082535/1082535 [==============================] - 164s 152us/sample - loss: 0.2891 - val_loss: 0.2902


In [15]:
y_pred = np.expm1(afm.predict(valid_inputs)).flatten()

In [16]:
cost(true=valid['price'], pred=y_pred)

0.5139392792610123